# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [146]:
import torch
import torch.nn as nn

embed = nn.Embedding(
    num_embeddings=20000,    # vocab size= 단어 사전의 단어 수 -> 총 몇 개 단어에 대한 embedding vector를 만들지
    embedding_dim=200,      # embedding vector의 차원 수 -> 개별 단어를 몇 개의 숫자(feature)로 표현할지
)

In [147]:
embed.weight.shape

torch.Size([20000, 200])

In [148]:
# embedding layer의 입력 - 문서를 구성하는 토큰들의 ID(정수여야 함!-int)를 1차원으로 묶어서 전달.

In [149]:
import torch
import torch.nn as nn

# 어휘사전 어휘 수: 10개
# 임베딩 벡터 차원 수: 3차원
e_layer = nn.Embedding(
    num_embeddings = 10,
    embedding_dim = 3
)
# 10 x 3 weight 행렬을 생성 -> weight 행렬이 전체 어휘들의 embidding vector들 

In [150]:
e_layer.weight

Parameter containing:
tensor([[-0.4045, -0.1565,  1.0083],
        [-0.6456, -0.3483, -0.5890],
        [ 1.4992,  0.0121, -0.5478],
        [ 0.1059, -0.8331,  0.3483],
        [ 0.0957, -0.8184,  1.2346],
        [ 0.1539,  1.5637, -0.3046],
        [-0.6230, -0.0877,  1.2724],
        [-1.2257, -1.6301,  1.4811],
        [ 0.1024, -1.4615, -0.3925],
        [-0.5138, -1.1440, -1.5887]], requires_grad=True)

`requires_grad=True`: 학습할 때는 "가중치"이지만, 학습이 끝나고 나면 "임베딩 벡터"가 된다~!

In [151]:
sent = "오늘 날씨 좋다" # ["오늘", "날씨", "좋다"]
# token = tokenizer.encode(sent).ids # 토큰화 # ["오늘", "날씨", "좋다"]
token = torch.tensor([0, 4, 2], dtype=torch.int64) # ["오늘":0, "날씨":4, "좋다":2]
e_layer(token)

tensor([[-0.4045, -0.1565,  1.0083],
        [ 0.0957, -0.8184,  1.2346],
        [ 1.4992,  0.0121, -0.5478]], grad_fn=<EmbeddingBackward0>)

# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

# Dataset, DataLoader 생성

## Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

## 데이터 로딩

In [8]:
import os
import time

from Korpora import Korpora

corpus = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\안수민\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\안수민\K

In [5]:
all_inputs = corpus.get_all_texts()     # inputs: 댓글들 전체
all_labels = corpus.get_all_labels()    # outputs: labels 전체 - 0: 부정, 1: 긍정
all_inputs[:5]
all_labels[:5]

len(all_inputs)

200000

In [6]:
corpus.train

NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]

In [7]:
corpus.test

NSMC.test: size=50000
  - NSMC.test.texts : list[str]
  - NSMC.test.labels : list[int]

In [8]:
corpus.test.texts[:5]

('굳 ㅋ',
 'GDNTOPCLASSINTHECLUB',
 '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
 '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
 '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??')

In [9]:
corpus.test.labels[:5]

[1, 0, 0, 0, 0]

## 토큰화
1. 형태소 단위 token화(분절)를 먼저 한다.
    - konlpy로 token화 한 뒤 다시 한 문장으로 만든다.
2. 1에서 처리한 corpus를 BPE 로 token화


형태소 분석 후 BPE를 쓰는 이유? 조사 분리 등 언어학적 정보와 서브워드 일반화를 모두 활용하려고.   
꼭 필요한가? ❌ BPE 하나만 사용해도 충분하다. 
   
### 전처리 함수

#### 형태소 단위 분절

In [152]:
from konlpy.tag import Okt
import string
import re

okt = Okt()

전처리 함수 정의

In [154]:
from konlpy.tag import Okt
import string
import re


def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화: okt.morphs()
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    """
    text = text.lower()# 구두점 제거 (stop word(불용어))
    text = re.sub(f"[{string.punctuation}]", " ", text)

    # 정규화
    tokens = okt.morphs(text, stem=True) # stem : 원형복원.
    return ' '.join(tokens)     # ["단어","단어",..] -> str "단어 단어 단어"

In [155]:
print(all_inputs[101])
text_preprocessing(all_inputs[101])

재미없다 진심 1 이훨 나 캐스팅 두 못 한 듯


'재미없다 진심 1 이훨 나 캐스팅 두 못 하다 듯'

전처리 함수에 문장들을 넣는다. 

In [156]:
s = time.time()
# train set 전처리
train_texts = corpus.train.texts
train_inputs = [text_preprocessing(txt) for txt in train_texts]
train_labels = corpus.train.labels

# test set 전처리
test_texts = corpus.test.texts
test_inputs = [text_preprocessing(txt) for txt in test_texts]
test_labels = corpus.test.labels
e = time.time()

print("전처리 걸린 시간(초):", e-s)


전처리 걸린 시간(초): 771.5486371517181


전처리한 trainset과 testset을 피클로 저장한다.   
딕셔너리 형태로 저장한다. 이때 input은 train_inputs=전처리한 문장 리스트이고, outputs는 train_labels=문장들의 라벨들(0, 1)이다.

In [ ]:
import pickle 
import os

os.makedirs("datasets/nsmc", exist_ok=True)
with open("datasets/nsmc/preprocessing_trainset.pkl", "wb") as fw:
    pickle.dump({"input":train_inputs, "outputs":train_labels}, fw)

with open("datasets/nsmc/preprocessing_testset.pkl", "wb") as fw:
    pickle.dump({"input":test_inputs, "outputs":test_labels}, fw)

In [ ]:
all_inputs = train_inputs + test_inputs

In [ ]:
len(train_inputs), len(test_inputs), len(all_inputs)

(150000, 50000, 200000)

pickle로 저장한 전처리한 데이터셋을 읽어온다.   
```python
with open("피클 파일 위치", "형식") as fr:
    dict = pickle.load(fr)
```

In [72]:
# pickle로 저장한 전처리한 데이터셋 읽어오기
import pickle

with open("datasets/nsmc/preprocessing_trainset.pkl", "rb") as fr:
    train_dic = pickle.load(fr)

with open("datasets/nsmc/preprocessing_testset.pkl", "rb") as fr:
    test_dic = pickle.load(fr)

In [73]:
train_inputs = train_dic['input']
train_labels = train_dic['outputs']

test_inputs = test_dic['input']
test_labels = test_dic['outputs']

In [74]:
all_inputs = train_inputs + test_inputs

In [75]:
len(train_inputs), len(test_inputs), len(all_inputs)

(150000, 50000, 200000)

### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
    - BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
    - https://huggingface.co/docs/tokenizers/quicktour
    - `pip install tokenizers`

In [76]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30_000 # 어휘사전 최대 단어 수
min_frequency = 5 # 사전에 추가할 최소 빈도수

tokenizer = Tokenizer(
    BPE(unk_token="[UNK]"), 
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size = vocab_size,
    min_frequency=min_frequency,
    special_tokens=["[PAD]", "[UNK]"],
    continuing_subword_prefix="##"
    # 단어의 중간에 나오는 subword일 경우 앞에 ##을 붙인다. 
    # "시작하는" -> "시작", "하는" => "시작", "##하는"
)
tokenizer.train_from_iterator(all_inputs, trainer=trainer) # vocab 생성 == tokenizer 학습

In [80]:
# 총 vocab size:
tokenizer.get_vocab_size()

26739

In [81]:
# 저장
os.makedirs("saved_models/nsmc", exist_ok=True)
tokenizer.save("saved_models/nsmc/tokenizer_bpe.json")

# load_tokenizer = Tokenizer.from_file("saved_models/nsmc/tokenizer_bpe.json")

In [82]:
idx = 1000
print(all_inputs[1000])
tokens = tokenizer.encode(all_inputs[idx])
print(tokens.ids)
print(tokens.tokens)

정말 최고 의 명작 성인 이 되다 보다 이집트 의 왕자 는 또 다른 감동 그 자체 네 요
[5420, 5438, 2203, 5530, 6570, 2206, 5425, 5410, 14758, 2203, 9123, 923, 1152, 5617, 5450, 651, 5641, 856, 2128]
['정말', '최고', '의', '명작', '성인', '이', '되다', '보다', '이집트', '의', '왕자', '는', '또', '다른', '감동', '그', '자체', '네', '요']


In [83]:
tokenizer.decode(tokens.ids)

'정말 최고 의 명작 성인 이 되다 보다 이집트 의 왕자 는 또 다른 감동 그 자체 네 요'

## Dataset, DataLoader 생성

In [21]:
# dset[0]
tokenizer.encode(train_inputs[0]).ids, train_labels[0]

([1986, 5881, 5426, 5667, 6087], 0)

In [22]:
# Pytorch 사용자 정의 Custom Dataset 정의
## 1. 데이터셋 상속
## 2. __len__(self): 총 데이터의 개수 반환
## 3. __getitem__(self, index): index의 x, y 반환



In [23]:
train_labels

[0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    def __init__(self, texts:list, labels, max_length, tokenizer):
        """
        texts: list - 댓글 목록. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
        labels: list - 댓글 Label(긍정/부정정) 목록. 
        max_length: 개별 댓글의 최대 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.labels = labels
        # self.texts: 입력 댓글 - token id로 변환된 댓글(문서). 글자수는 max_length에 맞춤
        #                        토큰이 max_length보다 적으면 [PAD] 추가, max_length보다 많으면 잘라낸다.
        self.texts = [self.__pad_token_sequences(tokenizer.encode(txt).ids) for txt in texts]

    ###########################################################################################
    # id로 구성된 개별 문장 token list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
    # max_length에 토큰리스트의 개수를 맞춰주는 함수.
    ############################################################################################
    def __pad_token_sequences(self, token_sequences):
        """
        id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
        max_length 보다 토큰수가 적으면 [PAD] 추가, 많으면 max_length 크기로 줄인다.
            ex) max_length = 5, [PAD] toekn id가 0
            - [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
            - [20, 30, 40, 50, 60, 70, 80][:5] => [20, 30, 40, 50, 60]
        """
        pad_token_id = self.tokenizer.token_to_id("[PAD]")
        seq_len = len(token_sequences) # 입력 받은 토큰 개수
        result = None
        if seq_len > self.max_length: # 잘라내기
            result = token_sequences[:self.max_length]
        else: 
            result = token_sequences + ([pad_token_id] * (self.max_length - seq_len)) # 추가할 값을 리스트에 넣어놓고 계산
        return result
        
    def __len__(self):
        return len(self.labels) # 총 데이터 개수를 반환.

    def __getitem__(self, idx):
        """
        idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
        Parameter
            idx: int 조회할 index
        Return
            tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
        """
        txt = self.texts[idx]
        label = self.labels[idx]

        return (torch.tensor(txt, dtype=torch.int64), torch.tensor([label], dtype=torch.float32))

In [85]:
tokenizer.token_to_id("[PAD]")

0

In [86]:
# 모든 댓글의 토큰 수를 조회
all_input_length = [len(tokenizer.encode(txt)) for txt in all_inputs]
all_input_length[:5]

[5, 11, 10, 9, 22]

In [27]:
import numpy as np
np.min(all_input_length), np.max(all_input_length)

(0, 89)

In [87]:
np.quantile(all_input_length, q=[0.9, 0.95])
# 전체 중 90%의 토큰 수는 29 미만, 95%는 41개 미만

array([29., 41.])

In [88]:
MAX_LENGTH = 30
trainset = NSMCDataset(train_inputs, train_labels, MAX_LENGTH, tokenizer)
testset = NSMCDataset(test_inputs, test_labels, MAX_LENGTH, tokenizer)

In [30]:
len(trainset), len(testset)

(150000, 50000)

In [31]:
trainset[10]

(tensor([  540, 11354,   506,  2408,  5414,  5426,  2408,  5414,   119,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor(1.))

위에서 0은 PAD를 의미. 

label 1은 긍정

데이터셋 만들었으니, 데이터 로더 만들어야겠죠?

In [89]:
BATCH_SIZE = 64
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE)

데이터 로더에 `len()` -> step 수

In [ ]:
len(train_loader), len(test_loader)

(2343, 782)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [90]:
import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [91]:
# 모델 정의
class NSMCClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, bidirectional=True, dropout_rate=0.2):
        """
        Args:
            vocab_size(int) - 어휘사전 총 어휘수
            embedding_dim(int) - word embedding vector의 차원수
            hidden_size(int) - LSTM의 hidden state의 feature 수
            num_layers(int) - LSTM의 layer의 개수
            bidirectional(bool) - LSTM의 양방향 여부
            dropout_rate(float) - LSTM이 두 개 이상의 layer로 구성된 경우 적용할 dropout 비율. 
                                  Dropout Layer의 dropout 비율
        """
        super().__init__()
        # 모델을 구성할 Layer들을 정의 - Embedding, LSTM, Dropout, Linear(추론기), Sigmoid
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,      # 총 단어 수 -> tokenizer에 등록된 총 단어수
            embedding_dim=embedding_dim,    # embedding vector의 차원 수
            padding_idx=0,                  # "[PAD]"의 토큰 ID(tokenizer.token_to_index("[PAD]")) 
                                            # 패딩 토큰은 학습하지 않는다. 

        )
        # embedding layer의 출력 shape: (batch_size:64, seq_length: 문서 토큰 수인 max_length 30, embedding_dim)

        self.lstm = nn.LSTM(
            input_size=embedding_dim,   # 개별 토큰(단어)의 feature 수 (embedding -> LSTM)
            hidden_size=hidden_size,    # 파라미터로 받은 hidden size
            num_layers=num_layers,      # 마찬가지로 파라미터로 받은 거 넣기
            bidirectional=bidirectional,
            dropout=dropout_rate if num_layers > 1 else 0,   # stacked rnn일 경우 설정.                                                    
                                                                         
        )

        self.dropout = nn.Dropout(dropout_rate) # LSTM과 Linear 사이에 과적합 방지를 위해서 사용한다. 

        # lstm의 출력: out, (hidden, cell)
        # out: 모든 타임스텝의 hidden state 값 # [seq_len, batch, hidden * bidirectional]
        # hidden: 마지막 타임스텝의 hidden state (단기기억) [bidirectional(2) * num_layers, batch, hidden]
        # cell: 마지막 타임스텝의 cell state (장기기억) 
        input_features = hidden_size * 2 if bidirectional else hidden_size
        self.classifier = nn.Linear(input_features, 1) # 출력 노드의 개수는 1: 이진분류 -> Positive일 확률
        self.sigmoid = nn.Sigmoid() # classifier의 출력값을 확률(0~1)값으로 변환하는 함수. 로지스틱

        

        
    def forward(self, X):
        """
        Args: 
            X(tensor): 입력 문서 토큰 리스트. shape: [batch_size, seq_length(max_length: 문서 구성 토큰 수)]
        """
        embedding_vectors = self.embedding(X)
        # [batch, seq_len] -> embedding -> [batch, seq_len, embedding 차원 수]
        # LSTM - batch_first=False: 입력 shape - [seq_len, batch_size, embedding_dim]
        # embedding_vectors의 batch 축과 seq_len >>축<<(값의 위치)를 바꿔준다. 
        embedding_vectors = embedding_vectors.transpose(1, 0) # 1번 축과 0번 축의 위치를 바꿔준다.
        out, _ = self.lstm(embedding_vectors)
        # out.shape: [seq_len, batch, hidden_size * (2 if bidirectional else 1)]
        # classifier(linear)에는 out의 마지막 index(마지막 seq) 값을 입력
        output = self.dropout(out[-1])
        output = self.classifier(out[-1])
        last_output = self.sigmoid(output)
        return last_output


        self.lstm()

[20, 30, 9, 0, 0] -> embedding layer -> [[0.1, 0.3, 0.7], [1.2, -0.4, 2.1], [0.3, 3.0, 1.2], [0, 0, 0], [0, 0, 0]]

padding_idx를 0으로 지정하면 -> 학습을 안 하기 때문에 -> embedding vector 확인 했을 때 0번 벡터는 학습이 안 된 채로 영벡터가 된다. 

(참고) `transpose()`와 `permute()`
- transpose는 축 두 개를, permute는 축 여러 개의 위치를 바꿀 때 쓴다. 

In [ ]:
# transpose()
# 코드 실행하지 말아용 필기한 거니까 🩷
ev.transpose(1, 0)
# 파라미터의 순서: index를 이동시킬 축 위치
# 파라미터 값: 이동할 대상 index의 축 위치
# 예) [2, 1] -> [1, 2]

ev = [
    [10, 20, 30],
    [40, 50, 60]
] # 2 x 3 tensor라면 # shape: (2, 3)
a = ev.transpose(1, 0)
# 10 idx: [0, 0] -> [0, 0]
# 20 idx: [0, 1] -> [1, 0]
# 30 idx: [0, 2] -> [2, 0]
# 40 idx: [1, 0] -> [0, 1]
a = [
    [10, 40],
    [20, 50],
    [30, 60]
]# shape: (3, 2)

b = a.reshape(3, 2) # 원소 순서대로 재배열
b = [
    [10, 20],
    [30, 40],
    [50, 60]
]

## 모델 생성

In [92]:
b = [a[0].max().item() for a in trainset]

In [93]:
max(b)

26738

In [94]:
tokenizer.get_vocab_size()

26739

In [95]:
VOCAB_SIZE = tokenizer.get_vocab_size()  # 총 어휘수
EMBEDDING_DIM = 100  # 하이퍼파라미터
HIDDEN_SIZE = 64
NUM_LAYERS = 2
BIDIRECTIONAL = True  
DROPOUT_RATE = 0.3

# 모델의 복잡도를 올린다. => EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS를 크게 잡으면 된다.
# Auto regressive(자기 회귀) 모형이 아니면 BIDIRECTIONAL = True

model = NSMCClassifier(
    vocab_size = VOCAB_SIZE,
    embedding_dim = EMBEDDING_DIM,
    hidden_size = HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    bidirectional=BIDIRECTIONAL,
    dropout_rate=DROPOUT_RATE
)
model = model.to(device)
print(model)

NSMCClassifier(
  (embedding): Embedding(26739, 100, padding_idx=0)
  (lstm): LSTM(100, 64, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [96]:
# summary
i = torch.randint(1, 10, (64, MAX_LENGTH)) # int64 타입의 dummy 입력 데이터터
# 입력 shape: (batch, seq_len)
summary(model, input_data=i, device=device)
# summary(모델, input_shape) => 내부적으로 입력데이터(float32)를 생성해서 추론론

Layer (type:depth-idx)                   Output Shape              Param #
NSMCClassifier                           [64, 1]                   --
├─Embedding: 1-1                         [64, 30, 100]             2,673,900
├─LSTM: 1-2                              [30, 64, 128]             184,320
├─Dropout: 1-3                           [64, 128]                 --
├─Linear: 1-4                            [64, 1]                   129
├─Sigmoid: 1-5                           [64, 1]                   --
Total params: 2,858,349
Trainable params: 2,858,349
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 525.03
Input size (MB): 0.02
Forward/backward pass size (MB): 3.50
Params size (MB): 11.43
Estimated Total Size (MB): 14.95

## 학습

### Train/Test 함수 정의

In [ ]:
# 1 epoch train 하는 함수
def train(model, dataloader, loss_fn, optimizer, device="cpu"):
    # 1. 모델을 train 모드로 변환
    model.train()
    # 2. 모델을 device로 이동
    model = model.to(device)
    total_loss = 0.0 # step별 loss를 저장.
    # step 단위로 모델 학습 (batch)
    # 데이터로더로부터 1 배치를 받아온다.
    for X, y in dataloader:
        # 1. X, y를 device로 이동
        X, y = X.to(device), y.to(device)
        # 2. 추론
        pred = model(X)
        # 3. loss 계산
        loss = loss_fn(pred, y) # 여기까지 순전파
        # 순전파에서 grad가 만들어짐 -> 역전파에서 쓴다.
        # 4. gradient 계산
        loss.backward()
        # 5. 파라미터 업데이트 w.data - w.grad * lr
        optimizer.step()
        # 6. gradient 초기화
        optimizer.zero_grad()
        # loss 누적
        total_loss += loss.item()
    # 1 epoch 학습 완료
    return total_loss / len(dataloader) # 1 epoch의 train loss를 반환. (total loss / step수)

In [ ]:
# 1 epoch 평가/검증 함수
def test(model, dataloader, loss_fn, device="cpu"):
    # 1. 모델을 eval 모드로 변경, 모델의 디바이스 이동
    model.eval()
    model = model.to(device)
    # loss, accuracy
    total_loss = 0.0
    total_acc = 0.0
    with torch.no_grad(): # 순전파할 때 grad(도함수)는 만들지 마라~
        for X, y in dataloader: 
            # 1 step 처리
            # 1. X, y를 device로 이동
            X, y = X.to(device), y.to(device)
            
            # 2. 추론
            pred_proba = model(X) # 양성일 확률 
            pred_label = (pred_proba).type(torch.int32) 
            total_loss += loss_fn(pred_proba, y).item()
            total_acc += (pred_label == y).sum().item() # [1]
        
        # return loss, acc
        return total_loss / len(dataloader), total_acc / len(dataloader.dataset)

[1]

1. `pred_label == y`: 정답을 맞혔나요? T/F
2. `(pred_label == y).sum()`: 정답을 맞춘 횟수
3. `item()`: tensor형이니까 뒤에 `item()` 붙여서 추출

### Train

In [99]:
LR = 0.0001
EPOCHS = 3
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [101]:
import time
s = time.time()
train_loss_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(EPOCHS):
    train_loss = train(model, train_loader, loss_fn, optimizer, device) # 1 epoch 학습
    val_loss, val_acc = test(model, test_loader, loss_fn, device)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)
    print(f"[{epoch}/{EPOCHS}] train loss: {train_loss}, val loss: {val_loss}, val acc: {val_acc}")

e = time.time()
print(e-s)

[0/3] train loss: 0.42536706964251225, val loss: 0.4136254567738689, val acc: 0.49654
[1/3] train loss: 0.38486712115267835, val loss: 0.39310799735357693, val acc: 0.49654
[2/3] train loss: 0.3605955367891187, val loss: 0.3972244799289557, val acc: 0.49654
499.6189196109772


## 모델저장

In [103]:
torch.save(model, "saved_models/nsmc/model.pt")

In [105]:
# 모델 로드
load_model = torch.load("saved_models/nsmc/model.pt", weights_only=False) # 모델 자체를 저장한 거야, weight만 저장한 게 아니라~!

# 서비스

## 전처리 함수들

In [128]:
from konlpy.tag import Okt
morph_tokenizer = Okt()

def text_preprocessing(text):
    
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]+", ' ', text)
    return ' '.join(morph_tokenizer.morphs(text, stem=True))

In [129]:
def pad_token_sequences(token_sequences, max_length):
    """padding 처리 메소드."""
    pad_token = tokenizer.token_to_id('[PAD]')  
    seq_length = len(token_sequences)           
    result = None
    if seq_length > max_length:                 
        result = token_sequences[:max_length]
    else:                                            
        result = token_sequences + ([pad_token] * (max_length - seq_length))
    return result

In [130]:
def predict_data_preprocessing(text_list):
    """
    모델에 입력할 수있는 input data를 생성
    Parameter:
        text_list: list - 추론할 댓글리스트
    Return
        torch.LongTensor - 댓글 token_id tensor
    """

    # cleansing + 정규화
    text_list = [text_preprocessing(txt) for txt in text_list]
    # text -> 토큰화화
    token_list = [tokenizer.encode(txt).ids for txt in text_list]
    # 토큰리스트의 크기(size)를 max_length에 맞추기
    token_list = [pad_token_sequences(token, MAX_LENGTH) for token in token_list]
    return torch.tensor(token_list, dtype=torch.int64)

## 추론

In [132]:
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]

In [133]:
import re

In [134]:
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

torch.Size([9, 30])

In [141]:
def predict(model, comment_list:list[str], input_tensor:torch.tensor, device="cpu"):
    """
    model로 comment_list(토큰id로 변환된 tensor)를 추론해서 긍정/부정적인 댓글인지 출력
    출력 형식
        comment(댓글) label 학률
        "아 재미없다."  부정  0.9 (부정적인 댓글일 확률)
        "재밌다."      긍정  0.87 (긍정적인 댓글일 확률)
    """
    model.eval()
    model = model.to(device)
    with torch.no_grad():
        pred = model(input_tensor) # shape: (batch, 1) -> positive일 확률
        for txt, pos_proba in zip(comment_list, pred):
            label = "긍정" if pos_proba.item() > 0.5 else "부정"
            proba = pos_proba.item() if pos_proba.item() > 0.5 else 1 - pos_proba.item() # 확률
            print(txt, label, round(proba, 3), sep="\t")

In [142]:
predict(model, comment_list, input_tensor, device)

아 진짜 재미없다.	부정	0.985
여기 식당 먹을만 해요	긍정	0.745
이걸 영화라고 만들었냐?	부정	0.925
기대 안하고 봐서 그런지 괜찮은데.	긍정	0.821
이걸 영화라고 만들었나?	부정	0.925
아! 뭐야 진짜.	부정	0.656
재미있는데.	긍정	0.98
연기 짱 좋아. 한번 더 볼 의향도 있다.	긍정	0.985
뭐 그럭저럭	부정	0.903


In [145]:
print("분석하려는 댓글을 입력하세요. 종료하려면 !quit을 입력하세요요")

while True:
    comment = input("댓글: ")
    if comment == "!quit":
        print("종료")
        break
    comment_list = [comment]
    input_tensor = predict_data_preprocessing([comment])
    predict(model, comment_list, input_tensor, device)

분석하려는 댓글을 입력하세요. 종료하려면 !quit을 입력하세요요
돈이 아까워	부정	0.987
대박이다	긍정	0.98
짜장면	긍정	0.55
우와 완전 못 만들었다.. 감동적이야..	긍정	0.971
ㅋ	긍정	0.735
종료
